In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


## Swoosh Detection

In [ ]:
'''
* 사용 모델
 YOLO : Swoosh 탐지 및 공정 분류 
'''

import cv2
from PIL import ImageFont, ImageDraw, Image
import numpy as np
import time
import os
from tensorflow.keras.models import load_model


# YOLO 설정 파일 Path
labelsPath = '../Swoosh-Realtime-Classifier/YOLO-configuration/custom.names'
weightsPath = '../Swoosh-Realtime-Classifier/YOLO-configuration/yolov4_final.weights'
configPath = '../Swoosh-Realtime-Classifier/YOLO-configuration/yolov4.cfg'

# YOLO 라벨(hand) 호출
YOLO_LABELS = open(labelsPath).read().strip().split("\n")

# YOLO 모델 호출
yolo_net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

# YOLO 출력층 설정
layer_names = yolo_net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in yolo_net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(YOLO_LABELS), 3))

# 실시간 웹캠 할당
cap = cv2.VideoCapture(0)

# 화면 폰트
font = cv2.FONT_HERSHEY_PLAIN
fonts = ImageFont.truetype(os.getcwd()+"\\fonts\\KoPubDotumBold.ttf", 32)

# FPS 측정 변수
starting_time = time.time()
frame_id = 0
count=0

while True:
    _, frame = cap.read()
    frame_id += 1
    height, width, channels = frame.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    yolo_net.setInput(blob)
    outs = yolo_net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.2:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                ww = int(detection[2] * width)
                hh = int(detection[3] * height)

                # Rectangle coordinates
                xx = int(center_x - ww / 2)
                yy = int(center_y - hh / 2)

                boxes.append([xx, yy, ww, hh])
                confidences.append(float(confidence))
                class_ids.append(class_id)

                #crop_img = frame[yy:yy + hh, xx:xx + hh]
                #cv2.imwrite('../Swoosh-Realtime-Classifier/img/swoosh_'+str(count)+".jpg", crop_img)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.8, 0.3)

    # 화면에 탐지 바운딩 박스 그리기
    for i in range(len(boxes)):
        if i in indexes:
            boxes = np.array(boxes)
            x, y, w, h = boxes[i]
            label = str(YOLO_LABELS[class_ids[i]])
            confidence = confidences[i]
            color = colors[class_ids[i]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, label + " " + str(round(confidence, 2)), (x, y + 30), font, 3, color, 3)

    # FPS 측정
    elapsed_time = time.time() - starting_time
    fps = frame_id / elapsed_time
    cv2.putText(frame, "FPS: " + str(round(fps, 2)), (10, 50), font, 4, (0, 0, 0), 3)

    # .mp4 파일 저장
    # vout.write(frame)

    # 디스플레이
    cv2.imshow("Image", frame)
    key = cv2.waitKey(1)

    # ESC 키로 종료
    if key == 27:
        break
        
cap.release()
cv2.destroyAllWindows()